# Set-Based Reinforcement Learning

In this notebook, we implement a set-based reinforcement learning algorithm, which is based on the paper [Training Verifiably Robust Agents using Set-Based Reinforcement Learning](https://arxiv.org/abs/2408.09112). 

In [1]:
import torch
import sys
sys.path.append('..')
from SBML import ZonoTorch as zt
from SBML import SBRL as sbrl

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

seed = torch.random.manual_seed(0)

System dynamics of 1d Quadrotor:
$$\begin{bmatrix}\dot z\\ \ddot z\end{bmatrix} = \begin{bmatrix}\dot z\\ (u+1)/(2*m)-g\end{bmatrix}$$
Reward function: 
$$r = -|z| - 0.1*|\dot z|$$

In [2]:
m = 0.05
g = 9.81

def dynamics(x,u):
    dx = torch.tensor([[x[0,1],(u[0,0]+1)/(2*m)-g]]).to(device=DEVICE)
    return dx

def reward(x,u,x_next):
    r = - torch.sum(torch.abs(x*torch.tensor([[1,0.1]]).to(device=DEVICE)))
    return r

init_state = zt.set.Interval(torch.tensor([[-4,4],[0,0]]).to(device=DEVICE))


Actor and Critic models are implemented using PyTorch. The actor and critic are simple feedforward neural networks with 2 hidden layers.

In [3]:
actor = torch.nn.Sequential(
    torch.nn.Linear(2, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 1),
    torch.nn.Tanh()
)

critic = torch.nn.Sequential(
    torch.nn.Linear(3, 64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 1)
)

In [4]:
env_options = {
    'ct': 0.1,
    'dt': 0.01,
    'max_step': 30,
    'initial_ops': 'uniform',
}

senv = sbrl.SetEnvironmnent(init_state,env_options,dynamics,reward,device=DEVICE)

ddpg_ops = {
    'actor_lr': 0.001,
    'actor_train_mode': 'set',
    'critic_lr': 0.01,
    'critic_train_mode': 'point',
    'gamma': 0.99,
    'tau': 0.005,
    'buffer_size': 1000,
    'batch_size': 64,
    'exp_noise': 0.1,
    'action_ub': 1,
    'action_lb': -1,
    'noise': .1,
    'actor_eta': 0.1,
}
agent = sbrl.DDPG(actor,critic,ddpg_ops,DEVICE)

In [5]:
agent.train(senv,1000,True)

Reinforcment Learning Parameters:
Standard-RL Options:
--------------------
Discount Factor (gamma): 0.99
Buffer Size: 1000
Batch Size: 64
Episodes: 1000
Device: cuda

Actor Options:
--------------
Learning Rate: 0.001
Training Mode: set
Eta: 0.1
Omega: 0.5
Noise: 0.1

Critic Options:
---------------
Learning Rate: 0.01
Training Mode: set
Eta: 0.01




RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn